In [1]:
import os
import glob
from joblib import Parallel, delayed
import pandas as pd
import numpy as np
import scipy as sc
from sklearn.model_selection import KFold
import lightgbm as lgb
import warnings
warnings.filterwarnings('ignore')
pd.set_option('max_columns', 300)

In [2]:
# data directory
data_dir = '../../../data/'

# Function to calculate first WAP
def calc_wap1(df):
    wap = (df['bid_price1'] * df['ask_size1'] + df['ask_price1'] * df['bid_size1']) / (df['bid_size1'] + df['ask_size1'])
    return wap

# Function to calculate second WAP
def calc_wap2(df):
    wap = (df['bid_price2'] * df['ask_size2'] + df['ask_price2'] * df['bid_size2']) / (df['bid_size2'] + df['ask_size2'])
    return wap

# Function to calculate third WAP
def calc_wap3(df):
    wap = (df['bid_price1'] * df['ask_size1'] + df['ask_price1']*df['bid_size1'] + df['bid_price2']*df['ask_size2'] + df['ask_price2']*df['bid_size2']) / (
                            df['ask_size1'] + df['bid_size1'] + df['ask_size2'] + df['bid_size2'])
    return wap

# Function to calculate the log of the return
# Remember that logb(x / y) = logb(x) - logb(y)
def log_return(series):
    return np.log(series).diff()

# Calculate the realized volatility
def realized_volatility(series):
    return np.sqrt(np.sum(series**2))
# Function to calculate the log of the return
# Remember that logb(x / y) = logb(x) - logb(y)
def log_return(series):
    return np.log(series).diff()

# Calculate the realized volatility
def realized_volatility(series):
    return np.sqrt(np.sum(series**2))

# Function to count unique elements of a series
def count_unique(series):
    return len(np.unique(series))

# Function to read our base train and test set
def read_train_test():
    train = pd.read_csv(data_dir + 'train.csv')
    test = pd.read_csv(data_dir + 'test.csv')
    # Create a key to merge with book and trade data
    train['row_id'] = train['stock_id'].astype(str) + '-' + train['time_id'].astype(str)
    test['row_id'] = test['stock_id'].astype(str) + '-' + test['time_id'].astype(str)
    print(f'Our training set has {train.shape[0]} rows')
    return train, test

def book_preprocessor(file_path):
    df = pd.read_parquet(file_path)
    # Calculate Wap
    df['wap1'] = calc_wap1(df)
    df['wap2'] = calc_wap2(df)
    df['wap3'] = calc_wap3(df)
    # Calculate log returns
    df['log_return1'] = df.groupby(['time_id'])['wap1'].apply(log_return)
    df['log_return2'] = df.groupby(['time_id'])['wap2'].apply(log_return)
    df['log_return3'] = df.groupby(['time_id'])['wap3'].apply(log_return)
    # Calculate wap balance
    df['wap_balance12'] = abs(df['wap1'] - df['wap2'])
    df['wap_balance13'] = abs(df['wap1'] - df['wap3'])
    df['wap_balance23'] = abs(df['wap2'] - df['wap3'])
    # Calculate spread
    df['price_spread'] = (df['ask_price1'] - df['bid_price1']) / ((df['ask_price1'] + df['bid_price1']) / 2)
    df['bid_spread'] = df['bid_price1'] - df['bid_price2']
    df['ask_spread'] = df['ask_price1'] - df['ask_price2']
    df['total_volume'] = (df['ask_size1'] + df['ask_size2']) + (df['bid_size1'] + df['bid_size2'])
    df['volume_imbalance'] = abs((df['ask_size1'] + df['ask_size2']) - (df['bid_size1'] + df['bid_size2']))
    # Dict for aggregations
    create_feature_dict = {
        'wap1': [np.sum, np.mean, np.std],
        'wap2': [np.sum, np.mean, np.std],
        'wap3': [np.sum, np.mean, np.std],
        'log_return1': [np.sum, realized_volatility, np.mean, np.std],
        'log_return2': [np.sum, realized_volatility, np.mean, np.std],
        'log_return3': [np.sum, realized_volatility, np.mean, np.std],
        'wap_balance12': [np.sum, np.mean, np.std],
        'wap_balance13': [np.sum, np.mean, np.std],
        'wap_balance23': [np.sum, np.mean, np.std],
        'price_spread':[np.sum, np.mean, np.std],
        'bid_spread':[np.sum, np.mean, np.std],
        'ask_spread':[np.sum, np.mean, np.std],
        'total_volume':[np.sum, np.mean, np.std],
        'volume_imbalance':[np.sum, np.mean, np.std]
    }
    
    # Function to get group stats for different windows (seconds in bucket)
    def get_stats_window(seconds_in_bucket, add_suffix = False):
        # Group by the window
        df_feature = df[df['seconds_in_bucket'] >= seconds_in_bucket].groupby(['time_id']).agg(create_feature_dict).reset_index()
        # Rename columns joining suffix
        df_feature.columns = ['_'.join(col) for col in df_feature.columns]
        # Add a suffix to differentiate windows
        if add_suffix:
            df_feature = df_feature.add_suffix('_' + str(seconds_in_bucket))
        return df_feature


    # Get the stats for different windows
    df_feature = get_stats_window(seconds_in_bucket = 0, add_suffix = False)
    df_feature_450 = get_stats_window(seconds_in_bucket = 450, add_suffix = True)
    df_feature_300 = get_stats_window(seconds_in_bucket = 300, add_suffix = True)
    df_feature_150 = get_stats_window(seconds_in_bucket = 150, add_suffix = True)
    
    # Merge all
    df_feature = df_feature.merge(df_feature_450, how = 'left', left_on = 'time_id_', right_on = 'time_id__450')
    df_feature = df_feature.merge(df_feature_300, how = 'left', left_on = 'time_id_', right_on = 'time_id__300')
    df_feature = df_feature.merge(df_feature_150, how = 'left', left_on = 'time_id_', right_on = 'time_id__150')
    # Drop unnecesary time_ids
    df_feature.drop(['time_id__450', 'time_id__300', 'time_id__150'], axis = 1, inplace = True)# Create row_id so we can merge
    stock_id = file_path.split('=')[1]
    df_feature['row_id'] = df_feature['time_id_'].apply(lambda x: f'{stock_id}-{x}')
    df_feature.drop(['time_id_'], axis = 1, inplace = True)
    return df_feature

In [3]:
stock_id = 0
file_path_book = data_dir + "book_train.parquet/stock_id=" + str(stock_id)
df = book_preprocessor(file_path_book)
df.head()

,wap1_sum,wap1_mean,wap1_std,wap2_sum,wap2_mean,wap2_std,wap3_sum,wap3_mean,wap3_std,log_return1_sum,log_return1_realized_volatility,log_return1_mean,log_return1_std,log_return2_sum,log_return2_realized_volatility,log_return2_mean,log_return2_std,log_return3_sum,log_return3_realized_volatility,log_return3_mean,log_return3_std,wap_balance12_sum,wap_balance12_mean,wap_balance12_std,wap_balance13_sum,wap_balance13_mean,wap_balance13_std,wap_balance23_sum,wap_balance23_mean,wap_balance23_std,price_spread_sum,price_spread_mean,price_spread_std,bid_spread_sum,bid_spread_mean,bid_spread_std,ask_spread_sum,ask_spread_mean,ask_spread_std,total_volume_sum,total_volume_mean,total_volume_std,volume_imbalance_sum,volume_imbalance_mean,volume_imbalance_std,wap1_sum_450,wap1_mean_450,wap1_std_450,wap2_sum_450,wap2_mean_450,wap2_std_450,wap3_sum_450,wap3_mean_450,wap3_std_450,log_return1_sum_450,log_return1_realized_volatility_450,log_return1_mean_450,log_return1_std_450,log_return2_sum_450,log_return2_realized_volatility_450,log_return2_mean_450,log_return2_std_450,log_return3_sum_450,log_return3_realized_volatility_450,log_return3_mean_450,log_return3_std_450,wap_balance12_sum_450,wap_balance12_mean_450,wap_balance12_std_450,wap_balance13_sum_450,wap_balance13_mean_450,wap_balance13_std_450,wap_balance23_sum_450,wap_balance23_mean_450,wap_balance23_std_450,price_spread_sum_450,price_spread_mean_450,price_spread_std_450,bid_spread_sum_450,bid_spread_mean_450,bid_spread_std_450,ask_spread_sum_450,ask_spread_mean_450,ask_spread_std_450,total_volume_sum_450,total_volume_mean_450,total_volume_std_450,volume_imbalance_sum_450,volume_imbalance_mean_450,volume_imbalance_std_450,wap1_sum_300,wap1_mean_300,wap1_std_300,wap2_sum_300,wap2_mean_300,wap2_std_300,wap3_sum_300,wap3_mean_300,wap3_std_300,log_return1_sum_300,log_return1_realized_volatility_300,log_return1_mean_300,log_return1_std_300,log_return2_sum_300,log_return2_realized_volatility_300,log_return2_mean_300,log_return2_std_300,log_return3_sum_300,log_return3_realized_volatility_300,log_return3_mean_300,log_return3_std_300,wap_balance12_sum_300,wap_balance12_mean_300,wap_balance12_std_300,wap_balance13_sum_300,wap_balance13_mean_300,wap_balance13_std_300,wap_balance23_sum_300,wap_balance23_mean_300,wap_balance23_std_300,price_spread_sum_300,price_spread_mean_300,price_spread_std_300,bid_spread_sum_300,bid_spread_mean_300,bid_spread_std_300,ask_spread_sum_300,ask_spread_mean_300,ask_spread_std_300,total_volume_sum_300,total_volume_mean_300,total_volume_std_300,volume_imbalance_sum_300,volume_imbalance_mean_300,volume_imbalance_std_300,wap1_sum_150,wap1_mean_150,wap1_std_150,wap2_sum_150,wap2_mean_150,wap2_std_150,wap3_sum_150,wap3_mean_150,wap3_std_150,log_return1_sum_150,log_return1_realized_volatility_150,log_return1_mean_150,log_return1_std_150,log_return2_sum_150,log_return2_realized_volatility_150,log_return2_mean_150,log_return2_std_150,log_return3_sum_150,log_return3_realized_volatility_150,log_return3_mean_150,log_return3_std_150,wap_balance12_sum_150,wap_balance12_mean_150,wap_balance12_std_150,wap_balance13_sum_150,wap_balance13_mean_150,wap_balance13_std_150,wap_balance23_sum_150,wap_balance23_mean_150,wap_balance23_std_150,price_spread_sum_150,price_spread_mean_150,price_spread_std_150,bid_spread_sum_150,bid_spread_mean_150,bid_spread_std_150,ask_spread_sum_150,ask_spread_mean_150,ask_spread_std_150,total_volume_sum_150,total_volume_mean_150,total_volume_std_150,volume_imbalance_sum_150,volume_imbalance_mean_150,volume_imbalance_std_150,row_id
0,303.125061,1.003725,0.000693,303.105539,1.003661,0.000781,303.125613,1.003727,0.000714,0.002292,0.004499,7.613599e-06,0.000260,0.002325,0.006999,0.000008,0.000404,0.002303,0.004106,7.650981e-06,0.000237,0.117051,0.000388,0.000295,0.059363,0.000197,0.000184,0.057688,0.000191,0.000200,0.257255,0.000852,0.000211,0.053006,0.000176,0.000162,-0.045557,-0.000151,0.000126,97696,323.496689,138.101214,40738,134.894040,107.260583,68.236749,1.003482,

In [9]:
vol_cols = []
time_variation = [1, 150, 300, 450]
# feature names using wap1, wap2, wap3
for i in range(1, 4):
    for t in time_variation:
        if t == 1:
            vol_cols += [f'log_return{i}_realized_volatility']
        else:
            vol_cols += [f'log_return{i}_realized_volatility_{t}']

# trade_features
for t in time_variation:
    if t == 1: 
        vol_cols += [f'trade_log_return_realized_volatility']
    else:
        vol_cols += [f'trade_log_return_realized_volatility_{t}']
    
for i in range(1, 4):
    print([col for col in vol_cols if f'log_return{i}' in col])
    
print([col for col in vol_cols if f'trade' in col])

['log_return1_realized_volatility', 'log_return1_realized_volatility_150', 'log_return1_realized_volatility_300', 'log_return1_realized_volatility_450']
['log_return2_realized_volatility', 'log_return2_realized_volatility_150', 'log_return2_realized_volatility_300', 'log_return2_realized_volatility_450']
['log_return3_realized_volatility', 'log_return3_realized_volatility_150', 'log_return3_realized_volatility_300', 'log_return3_realized_volatility_450']
['trade_log_return_realized_volatility', 'trade_log_return_realized_volatility_150', 'trade_log_return_realized_volatility_300', 'trade_log_return_realized_volatility_450']
